In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
import math
import warnings
from sqlalchemy import *

In [4]:
#To Ignore the warnings
warnings.filterwarnings("ignore")

In [7]:
#Load the data
data = pd.read_csv("taxi_fare.csv")

In [9]:
#Check the data shape and type
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212345 entries, 0 to 212344
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               212345 non-null  int64  
 1   tpep_pickup_datetime   212345 non-null  object 
 2   tpep_dropoff_datetime  212345 non-null  object 
 3   passenger_count        212345 non-null  int64  
 4   pickup_longitude       212345 non-null  float64
 5   pickup_latitude        212345 non-null  float64
 6   RatecodeID             212345 non-null  int64  
 7   store_and_fwd_flag     212345 non-null  object 
 8   dropoff_longitude      212345 non-null  float64
 9   dropoff_latitude       212345 non-null  float64
 10  payment_type           212345 non-null  int64  
 11  fare_amount            212345 non-null  float64
 12  extra                  212345 non-null  float64
 13  mta_tax                212345 non-null  float64
 14  tip_amount             212345 non-nu

In [11]:
#Check for NULL
print(data.isnull().sum())

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
pickup_longitude         0
pickup_latitude          0
RatecodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64


In [13]:
#Drop Duplicates
data.drop_duplicates(inplace=True)

In [15]:
#Drop rows where latitude and longitudes are zero and total amount is less than zero
data = data[(data['pickup_latitude']!=0) & (data['pickup_longitude']!=0) & (data['dropoff_latitude']!=0) & (data['dropoff_longitude']!=0)
data.reset_index(drop=True,inplace=True)

In [17]:
#Create a function to calculate the distance using Haversine Formula
def haversine(lat1, lon1, lat2, lon2):
    
    # distance between latitudes
    # and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0

    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [19]:
for i in range(0,len(data)):
    data.loc[i,'trip_distance']=haversine(data['pickup_latitude'][i],
                                            data['pickup_longitude'][i],
                                            data['dropoff_latitude'][i],
                                            data['dropoff_longitude'][i])

In [21]:
#Drop rows where trip distance is zero
data = data[data['trip_distance']>0]
data.reset_index(drop=True,inplace=True)

In [23]:
#Drop the rows where passenger count is zero
data = data[data['passenger_count']>0]
data.reset_index(drop=True,inplace=True)

In [35]:
#Function to convert the date column to UTC and to EDT
def utc_est(date):
    est = pytz.timezone('US/Eastern')
    utc = pd.to_datetime(date,utc=True)
    est_time = utc.astimezone(est)
    return est_time

In [37]:
#Change the date time column to EDT
for i in range(0,len(data)):
    data.loc[i,'Pickup_date']=utc_est(data['tpep_pickup_datetime'][i])
    data.loc[i,'Dropoff_date']=utc_est(data['tpep_dropoff_datetime'][i])

In [41]:
#Delete the rows where Pick up time and Drop off time is same
data = data[data['Pickup_date']!=data['Dropoff_date']]
data.reset_index(drop=True,inplace=True)

In [43]:
#create a new column week_day which represents the day of the week
# Values - - - > Monday = 1, Tuesay = 2,.....Sunday = 7
for i in range(0,len(data)):
    data.loc[i,'day_of_week'] = data['Pickup_date'][i].isoweekday()

In [45]:
data = data.astype({'day_of_week': int})

In [47]:
#Check Whether the pick up time is AM/PM and create a new column 'AM/PM'
for i in range(0,len(data)):
    if data['Pickup_date'][i].hour < 12:
        data.loc[i,'AM/PM'] = 'AM'
    else:
        data.loc[i,'AM/PM'] = 'PM'

In [51]:
#Calculate the Trip Duration
for i in range(0,len(data)):
    d1 = data['Dropoff_date'][i].to_pydatetime() - data['Pickup_date'][i].to_pydatetime()
    d1 = divmod(d1.total_seconds(),60)
    data.loc[i,'Trip_duration_minutes'] = d1[0]+(d1[1]/100)

In [53]:
for i in range(0,len(data)):
    data.loc[i,'Pickup_time'] = data['Pickup_date'][i].time().hour

In [55]:
#Change the Pick up time to int
data = data.astype({'Pickup_time': int})

In [67]:
#Drop where the total amount is less than zero
data = data[data['total_amount']>0]
data.reset_index(drop=True,inplace=True)

In [68]:
# ## Create SQLAlchemy engine to create the data base
username = "root"
password = "Pgnkka#"
host = "localhost"
port = 3306
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}")
create_db_query = """
                     CREATE DATABASE Taxi_ride_fare
                     """
with engine.connect() as conn:
    conn.execute(text(create_db_query))
    conn.commit()

In [85]:
#insert the data to the SQL database
database = "Taxi_ride_fare"
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")
data.to_sql("Taxi_fare",engine,index=False)

208693